# callable

### 함수를 정의해서 callable 여부 확인

In [1]:
from types import FunctionType as function

def f(func):
    #if not isinstance(func, function):
    #    return False
    if not callable(func):
        return False
    # login  
    return func

In [2]:
#### 특정 함수를 넣어서 callabe 여부 체크

In [4]:
is_callable = [ f, type, sum, 0, 'str', lambda x : x+1 ]

In [5]:
[ *map(callable, is_callable) ]

[True, True, True, False, False, True]

### callable 함수를 알아보기



In [6]:
def callable_check(object):
    return hasattr(object, '__call__')

# O

In [7]:
callable_check(f)

True

#### 클래스의 인스턴스도 호출이 가능 여부 확인

In [8]:
class C:
    def __call__(self):
        pass

In [9]:
c = C()

In [10]:
callable_check(c)

True

In [15]:
class Fc:
    def __call__(self):
        raise TypeError("error")

In [16]:
fc = Fc()

In [17]:
try :
    fc()
except Exception as e :
    print(e)

error


In [18]:
callable(fc)

True

#### 스페셜 메소드 이름만 사용 할 경우

In [19]:
class Fc2:
    __call__ = 3

In [20]:
fc2 = Fc2()

In [21]:
callable(fc2)

True

In [22]:
class Fi:
    __iter__ = 123
    # def __iter__(self):
    #    pass

In [23]:
fi = Fi()

In [24]:
from collections import Iterable

In [25]:
isinstance(fi, Iterable)

True

### 함수 매개변수 초기값 처리 알아보기

In [27]:
def f(x):
    print(x)
    return x

In [28]:
def t(a=[f(x) for x in range(3)]):
    pass

0
1
2


In [29]:
t.__defaults__

([0, 1, 2],)

#### 초기값에 리스트 등 변경가능한 것이 들어가면 안되는 이유

In [30]:
def f(x=[]):
    x.append(3)
    return x

In [31]:
f() # 3만 들어있는 리스트를 받고싶다ㅣ

[3]

In [32]:
f() # 또 3만 들어있는 리스트를 받고싶다

[3, 3]

In [33]:
f.__defaults__

([3, 3],)

In [34]:
def f(x=None):
    if not x:
        return [3]
    return x.append(3)

In [35]:
f()

[3]

In [36]:
f()

[3]

In [37]:
f()

[3]

In [39]:
f.__defaults__

(None,)

### 사용자 함수 이해하기 : all, any

In [40]:
def all_(values): # True 추정
    for value in values:
        if not value:
            return False
    return True

In [42]:
all([])

True

In [43]:
all_([])

True

In [48]:
def any_(values): # False추정
    for value in values:
        if value:
            return True
    return False

In [44]:
any([])

False

In [49]:
any_([])

False

#### 절사처리 

In [88]:
round(1.5)

2

In [51]:
round(1.5,1)

1.5

In [54]:
import math

In [55]:
math.floor(1.5)

1

In [56]:
math.ceil(1.5)

2

In [90]:
round(2.5)

2

In [52]:
round(2.5,1)

2.5

###  클로저 환경 이해하기

In [57]:
# None closure
def outer():
    def inner():
        b = 3
    return inner

inner = outer()


In [62]:
inner.__closure__

In [59]:
inner.__code__.co_varnames

('b',)

In [60]:
inner.__code__.co_freevars

()

In [63]:
# closure
def outer():
    a = 'abc'
    x = 3
    def inner():
        b = x * a 
    return inner

# outer 를 탐색 X , inner 를 탐색할수있게됩니다.

In [64]:
inner = outer()

In [65]:
inner.__closure__

(<cell at 0x000000000662C258: str object at 0x0000000001DF4D50>,
 <cell at 0x000000000662C558: int object at 0x0000000055D86C50>)

In [66]:
cell0 = inner.__closure__[0]

In [67]:
cell1 = inner.__closure__[1]

In [68]:
cell0.cell_contents,  cell1.cell_contents

('abc', 3)

In [69]:
inner.__code__.co_varnames

('b',)

In [70]:
inner.__code__.co_freevars

('a', 'x')

## 함수 종료
#### 함수안에 값들은 del
#### python gc : 참조가 끊겼거나, 혹은 참조할 수 없는 상태

In [71]:
def outer(func):
    def inner(*args, **kwargs):
        return func(*args, **kwargs)
    return inner

In [72]:
@outer
def test():
    print("test")

In [73]:
test.__name__

'inner'

In [74]:
# test() 한다는걸 결국 inner()

In [215]:
# 입력받은 값들의 평균을 구하는 함수
# 입력받는 값은 숫자, 1개만
# 여러번 호출되면, 그동안 입력받은 숫자들의 평균을 구하는 함수

In [75]:
def make_averager():
    values = []
    def averager(n):
        values.append(n)
        return sum(values) / len(values)
    return averager

In [76]:
averager = make_averager()

In [77]:
averager(10)

10.0

In [78]:
averager(11)

10.5

In [79]:
averager(12)

11.0

In [80]:
averager.__closure__[0].cell_contents

[10, 11, 12]

In [81]:
def averager(n, values=[]):
    values.append(n)
    return sum(values) / len(values)

In [82]:
averager(10)

10.0

In [83]:
averager(11)

10.5

In [84]:
averager(12)

11.0

In [85]:
averager.__defaults__

([10, 11, 12],)

In [227]:
# 가장 빠른 함수를 알려주는 데코레이터
# 여러함수들의 실행시간을 비교해서, 가장 빠른 함수를 알려주는 데코레이터

In [86]:
import time


def make_time_checker(log=None):
    if not log:
        log = {} # 함수이름 key, 함수실행시간을 value 
    def time_check(func):
        def inner(*args, **kwargs):
            start_time = time.perf_counter()
            result = func(*args, **kwargs)
            spend = time.perf_counter() - start_time
            
            log[func.__name__] = spend # 함수 실행 결과(시간) 저장
            
            best = min(log, key=lambda x : log[x] )
            
            print("best : {} / {}".format(
                best, 
                log[best] # 실행시간
            ))
            
        return inner
    return time_check

In [87]:
time_checker = make_time_checker()

In [88]:
@time_checker
def sum1(n):
    return sum(range(n))


@time_checker
def sum2(n):
    result = 0
    for i in range(n):
        result += i
    return result


@time_checker
def sum3(n):
    result = 0
    i = 0
    while True:
        result += i
        i += 1
        if i > N:
            break
    return result

In [89]:
N = 100000

In [90]:
sum1(N)

best : sum1 / 0.004025841924398625


In [91]:
sum2(N)

best : sum1 / 0.004025841924398625


In [92]:
sum3(N)

best : sum1 / 0.004025841924398625


In [93]:
class Averager:
    values = []
    def __call__(self, n):
        values = Averager.values # type(self).values 
        # self.values
        values.append(n)
        return sum(values) / len(values)

In [94]:
averager = Averager()
averager1 = Averager()

In [95]:
averager(10)

10.0

In [96]:
averager(11)

10.5

In [97]:
averager(12)

11.0

In [98]:
averager1(30)

15.75

In [99]:
import inspect

In [100]:
print(inspect.getsource(sum1))

        def inner(*args, **kwargs):
            start_time = time.perf_counter()
            result = func(*args, **kwargs)
            spend = time.perf_counter() - start_time
            
            log[func.__name__] = spend # 함수 실행 결과(시간) 저장
            
            best = min(log, key=lambda x : log[x] )
            
            print("best : {} / {}".format(
                best, 
                log[best] # 실행시간
            ))



In [102]:
try :
    inspect.getsource(print)
except Exception as e :
    print(e)

<built-in function print> is not a module, class, method, function, traceback, frame, or code object


In [295]:
# inspect.getmembers

In [103]:
dict(inspect.getmembers(averager))['__call__']

<bound method Averager.__call__ of <__main__.Averager object at 0x0000000006648320>>

In [104]:
inspect.getmembers(averager, inspect.ismethod)

[('__call__',
  <bound method Averager.__call__ of <__main__.Averager object at 0x0000000006648320>>)]

In [106]:
inspect.isfunction

<function inspect.isfunction>

In [107]:
inspect.isclass

<function inspect.isclass>

### 문자열 문장을 직접 런타임에 실행하기

In [108]:
ast_m = """
a = 3
b = 'a'
c = a * b
"""

In [109]:
code = compile(ast_m, '', 'exec')

In [110]:
exec(code)

In [111]:
a

3

In [112]:
b

'a'

In [113]:
c

'aaa'

In [115]:
python_code = """
a_c = [ x for x in range(1, 10+1)]
"""

python_code2 = """
a_l = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
"""

In [117]:
exec(python_code)

In [118]:
a_c

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [119]:
exec(python_code2)

In [120]:
a_l

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]